## import modules

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Activation
from tensorflow.keras.optimizers import SGD, Adagrad, Adadelta, RMSprop, Adam
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore')

In [ ]:
from matplotlib.pyplot import imshow
from keras.preprocessing import image
from keras import applications
import os
import glob

## Loading the dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/project/archive (1).zip'

Streaming output truncated to the last 5000 lines.
  inflating: cropped/train/n02102973-Irish_water_spaniel/n02102973_1066.jpg  
  inflating: cropped/train/n02102973-Irish_water_spaniel/n02102973_1140.jpg  
  inflating: cropped/train/n02102973-Irish_water_spaniel/n02102973_1200.jpg  
  inflating: cropped/train/n02102973-Irish_water_spaniel/n02102973_1270.jpg  
  inflating: cropped/train/n02102973-Irish_water_spaniel/n02102973_1299.jpg  
  inflating: cropped/train/n02102973-Irish_water_spaniel/n02102973_1318.jpg  
  inflating: cropped/train/n02102973-Irish_water_spaniel/n02102973_1409.jpg  
  inflating: cropped/train/n02102973-Irish_water_spaniel/n02102973_158.jpg  
  inflating: cropped/train/n02102973-Irish_water_spaniel/n02102973_1615.jpg  
  inflating: cropped/train/n02102973-Irish_water_spaniel/n02102973_1622.jpg  
  inflating: cropped/train/n02102973-Irish_water_spaniel/n02102973_1669.jpg  
  inflating: cropped/train/n02102973-Irish_water_spaniel/n02102973_1714.jpg  
  inflating: c

## Analyze and Visualize the Dataset

In [ ]:
img = cv2.imread('/content/cropped/cropped/train/n02085620-Chihuahua/n02085620_10621.jpg')
print(img.shape)
img1 = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
plt.imshow(img1)

In [ ]:
img = cv2.imread('/content/cropped/cropped/test/n02085620-Chihuahua/n02085620_10621.jpg')
print(img.shape)
img1 = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
plt.imshow(img1)

## Preprocessing

In [ ]:
# Define the data generators with preprocessing steps
train_gen = ImageDataGenerator(
    rescale=1./255,            # Normalize pixel values to [0, 1]
    horizontal_flip=True       # Randomly flip images horizontally
)

test_gen = ImageDataGenerator(rescale=1./255)  # Only rescale for test set

In [ ]:
# Load data
train_data = train_gen.flow_from_directory(
    '/content/cropped/cropped/train',  # Path to training data directory
    target_size=(224, 224),              # Resize images to 224x224
    batch_size=32,
    class_mode='categorical'                  # Set class mode to binary (or categorical based on your problem)
)

test_data = test_gen.flow_from_directory(
    '/content/cropped/cropped/test',   # Path to testing data directory
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 12000 images belonging to 120 classes.
Found 8580 images belonging to 120 classes.


In [ ]:
train_data.class_indices

{'n02085620-Chihuahua': 0,
 'n02085782-Japanese_spaniel': 1,
 'n02085936-Maltese_dog': 2,
 'n02086079-Pekinese': 3,
 'n02086240-Shih-Tzu': 4,
 'n02086646-Blenheim_spaniel': 5,
 'n02086910-papillon': 6,
 'n02087046-toy_terrier': 7,
 'n02087394-Rhodesian_ridgeback': 8,
 'n02088094-Afghan_hound': 9,
 'n02088238-basset': 10,
 'n02088364-beagle': 11,
 'n02088466-bloodhound': 12,
 'n02088632-bluetick': 13,
 'n02089078-black-and-tan_coonhound': 14,
 'n02089867-Walker_hound': 15,
 'n02089973-English_foxhound': 16,
 'n02090379-redbone': 17,
 'n02090622-borzoi': 18,
 'n02090721-Irish_wolfhound': 19,
 'n02091032-Italian_greyhound': 20,
 'n02091134-whippet': 21,
 'n02091244-Ibizan_hound': 22,
 'n02091467-Norwegian_elkhound': 23,
 'n02091635-otterhound': 24,
 'n02091831-Saluki': 25,
 'n02092002-Scottish_deerhound': 26,
 'n02092339-Weimaraner': 27,
 'n02093256-Staffordshire_bullterrier': 28,
 'n02093428-American_Staffordshire_terrier': 29,
 'n02093647-Bedlington_terrier': 30,
 'n02093754-Border_terr

In [ ]:
# Calculate class weights
y_train = train_data.classes
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights))
print(class_weights)

{0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0, 4: 1.0, 5: 1.0, 6: 1.0, 7: 1.0, 8: 1.0, 9: 1.0, 10: 1.0, 11: 1.0, 12: 1.0, 13: 1.0, 14: 1.0, 15: 1.0, 16: 1.0, 17: 1.0, 18: 1.0, 19: 1.0, 20: 1.0, 21: 1.0, 22: 1.0, 23: 1.0, 24: 1.0, 25: 1.0, 26: 1.0, 27: 1.0, 28: 1.0, 29: 1.0, 30: 1.0, 31: 1.0, 32: 1.0, 33: 1.0, 34: 1.0, 35: 1.0, 36: 1.0, 37: 1.0, 38: 1.0, 39: 1.0, 40: 1.0, 41: 1.0, 42: 1.0, 43: 1.0, 44: 1.0, 45: 1.0, 46: 1.0, 47: 1.0, 48: 1.0, 49: 1.0, 50: 1.0, 51: 1.0, 52: 1.0, 53: 1.0, 54: 1.0, 55: 1.0, 56: 1.0, 57: 1.0, 58: 1.0, 59: 1.0, 60: 1.0, 61: 1.0, 62: 1.0, 63: 1.0, 64: 1.0, 65: 1.0, 66: 1.0, 67: 1.0, 68: 1.0, 69: 1.0, 70: 1.0, 71: 1.0, 72: 1.0, 73: 1.0, 74: 1.0, 75: 1.0, 76: 1.0, 77: 1.0, 78: 1.0, 79: 1.0, 80: 1.0, 81: 1.0, 82: 1.0, 83: 1.0, 84: 1.0, 85: 1.0, 86: 1.0, 87: 1.0, 88: 1.0, 89: 1.0, 90: 1.0, 91: 1.0, 92: 1.0, 93: 1.0, 94: 1.0, 95: 1.0, 96: 1.0, 97: 1.0, 98: 1.0, 99: 1.0, 100: 1.0, 101: 1.0, 102: 1.0, 103: 1.0, 104: 1.0, 105: 1.0, 106: 1.0, 107: 1.0, 108: 1.0, 109: 1.0, 110: 1.0,

## ResNet101

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define constants
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
TRAIN_PATH = '/content/cropped/cropped/train'
VAL_PATH = '/content/cropped/cropped/test'

# Use ImageDataGenerator for data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 32 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Flow validation images in batches of 32 using val_datagen generator
val_generator = val_datagen.flow_from_directory(
    VAL_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Number of classes
num_classes = len(train_generator.class_indices)
print("Number of classes:", num_classes)


Found 12000 images belonging to 120 classes.
Found 8580 images belonging to 120 classes.
Number of classes: 120


In [ ]:
# Load ResNet-101 model pre-trained on ImageNet
base_model = tf.keras.applications.ResNet101(
    include_top=False,
    weights='imagenet',
    input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)
)

# Freeze the base model layers
base_model.trainable = False

# Add classification head to the base model
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


171446536/171446536 [==============================] - 1s 0us/step
Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet101 (Functional)      (None, 7, 7, 2048)        42658176  
                                                                 
 global_average_pooling2d_3  (None, 2048)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_20 (Dense)            (None, 1024)              2098176   
                                                                 
 dropout_2 (Dropout)         (None, 1024)              0         
                                                                 
 dense_21 (Dense)            (None, 120)               123000    
                                                                 
Total params: 44879352 (171.20 MB)
Trainable params: 

In [ ]:
epochs = 20

history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=len(val_generator)
)


In [ ]:
val_loss, val_acc = model.evaluate(val_generator, steps=len(val_generator))
print(f"Validation accuracy: {val_acc:.4f}")


## ResNet50

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define constants
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
TRAIN_PATH = '/content/cropped/cropped/train'
VAL_PATH = '/content/cropped/cropped/test'

# Use ImageDataGenerator for data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 32 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Flow validation images in batches of 32 using val_datagen generator
val_generator = val_datagen.flow_from_directory(
    VAL_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Number of classes
num_classes = len(train_generator.class_indices)
print("Number of classes:", num_classes)


Found 12000 images belonging to 120 classes.
Found 8580 images belonging to 120 classes.
Number of classes: 120


In [ ]:
# Load ResNet-50 model pre-trained on ImageNet
base_model = tf.keras.applications.ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)
)

# Freeze the base model layers
base_model.trainable = False

# Add classification head to the base model
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


94765736/94765736 [==============================] - 1s 0us/step
Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d_4  (None, 2048)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_22 (Dense)            (None, 1024)              2098176   
                                                                 
 dropout_3 (Dropout)         (None, 1024)              0         
                                                                 
 dense_23 (Dense)            (None, 120)               123000    
                                                                 
Total params: 25808888 (98.45 MB)
Trainable params: 222

In [ ]:
epochs = 20

history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=len(val_generator)
)

In [ ]:
val_loss, val_acc = model.evaluate(val_generator, steps=len(val_generator))
print(f"Validation accuracy: {val_acc:.4f}")

## Inception ResNetv2

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define constants
IMG_SIZE = (224, 224)  # InceptionResNetV2 requires input size of (299, 299)
BATCH_SIZE = 32
TRAIN_PATH = '/content/cropped/cropped/train'
VAL_PATH = '/content/cropped/cropped/test'

# Use ImageDataGenerator for data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 32 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Flow validation images in batches of 32 using val_datagen generator
val_generator = val_datagen.flow_from_directory(
    VAL_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Number of classes
num_classes = len(train_generator.class_indices)
print("Number of classes:", num_classes)


Found 12000 images belonging to 120 classes.
Found 8580 images belonging to 120 classes.
Number of classes: 120


In [ ]:
# Load InceptionResNetV2 model pre-trained on ImageNet
base_model = tf.keras.applications.InceptionResNetV2(
    include_top=False,
    weights='imagenet',
    input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)
)

# Freeze the base model layers
base_model.trainable = False

# Add classification head to the base model
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

219055592/219055592 [==============================] - 3s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_resnet_v2 (Funct  (None, 5, 5, 1536)        54336736  
 ional)                                                          
                                                                 
 global_average_pooling2d_1  (None, 1536)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_1 (Dense)             (None, 1024)              1573888   
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_2 (Dense)             (None, 120)               123000    
                                                       

In [ ]:
# Load InceptionResNetV2 model pre-trained on ImageNet
base_model = tf.keras.applications.InceptionResNetV2(
    include_top=False,
    weights='imagenet',
    input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)
)

# Freeze the base model layers
base_model.trainable = False

# Add classification head to the base model
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_resnet_v2 (Funct  (None, 5, 5, 1536)        54336736  
 ional)                                                          
                                                                 
 global_average_pooling2d_2  (None, 1536)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_3 (Dense)             (None, 1024)              1573888   
                                                                 
 dropout_1 (Dropout)         (None, 1024)              0         
                                                                 
 dense_4 (Dense)             (None, 120)               123000    
                                                                 
Total params: 56033624 (213.75 MB)
Trainable params: 1

In [ ]:
epochs = 20

history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=len(val_generator)
)
history = model.fit(train_set,
                    epochs=30,
                    steps_per_epoch=len(train_set),
                    validation_data=test_set,verbose=1,callbacks=callback)

Epoch 1/20
375/375 [==============================] - 234s 584ms/step - loss: 1.1475 - accuracy: 0.7214 - val_loss: 0.4302 - val_accuracy: 0.8749
Epoch 2/20
375/375 [==============================] - 214s 571ms/step - loss: 0.7948 - accuracy: 0.7860 - val_loss: 0.4217 - val_accuracy: 0.8777
Epoch 3/20
375/375 [==============================] - 213s 567ms/step - loss: 0.7603 - accuracy: 0.7942 - val_loss: 0.3966 - val_accuracy: 0.8798
Epoch 4/20
375/375 [==============================] - 215s 574ms/step - loss: 0.7407 - accuracy: 0.7952 - val_loss: 0.3932 - val_accuracy: 0.8832
Epoch 5/20
375/375 [==============================] - 216s 576ms/step - loss: 0.7454 - accuracy: 0.7968 - val_loss: 0.4001 - val_accuracy: 0.8839
Epoch 6/20
375/375 [==============================] - 216s 575ms/step - loss: 0.7247 - accuracy: 0.7989 - val_loss: 0.3832 - val_accuracy: 0.8896
Epoch 7/20
375/375 [==============================] - 254s 677ms/step - loss: 0.7158 - accuracy: 0.8017 - val_loss: 0.3957 -

In [ ]:
val_loss, val_acc = model.evaluate(val_generator, steps=len(val_generator))
print(f"Validation accuracy: {val_acc:.4f}")

## Inception V3

In [ ]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model,load_model
from glob import glob
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense,Activation,GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf

In [ ]:
inception_model =InceptionV3(include_top=False,input_shape=[224,224,3])

87910968/87910968 [==============================] - 0s 0us/step


In [ ]:
for layer in inception_model.layers:
    layer.trainable = False

In [ ]:
x =GlobalAveragePooling2D()(inception_model.output)

x = Dense(120)(x)

pred = Activation('softmax')(x)

In [ ]:
model = Model(inputs=inception_model.input,outputs=pred)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 111, 111, 32)         864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 111, 111, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 111, 111, 32)         0         ['batch_normalization[0][0

In [ ]:
train_datagen =  ImageDataGenerator(rescale=1./255,
                                   shear_range=0.15,
                                   rotation_range=20,
                                   width_shift_range=0.1,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest',
                                   height_shift_range=0.1
                                   )
test_datagen = ImageDataGenerator(rescale=1./255)


train_set = train_datagen.flow_from_directory('/content/cropped/cropped/train',
                                              target_size=(224,224),
                                              batch_size=32,
                                              class_mode='categorical'
                                             )

test_set = train_datagen.flow_from_directory('/content/cropped/cropped/test',
                                              target_size=(224,224),
                                              batch_size=32,
                                              class_mode='categorical'
                                             )

Found 12000 images belonging to 120 classes.
Found 8580 images belonging to 120 classes.


In [ ]:
checkpoint = ModelCheckpoint('smart_model_mobile.h5',monitor='val_accuracy',verbose=1,save_best_only=True)
callback = [checkpoint]

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history = model.fit(train_set,epochs=10,steps_per_epoch=len(train_set),validation_data=test_set,verbose=1,callbacks=callback)

Epoch 1/10
375/375 [==============================] - ETA: 0s - loss: 0.9036 - accuracy: 0.7502
Epoch 1: val_accuracy improved from -inf to 0.78986, saving model to smart_model_mobile.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


375/375 [==============================] - 268s 713ms/step - loss: 0.9036 - accuracy: 0.7502 - val_loss: 0.7237 - val_accuracy: 0.7899
Epoch 2/10
375/375 [==============================] - ETA: 0s - loss: 0.6454 - accuracy: 0.8048
Epoch 2: val_accuracy did not improve from 0.78986
375/375 [==============================] - 280s 747ms/step - loss: 0.6454 - accuracy: 0.8048 - val_loss: 0.7917 - val_accuracy: 0.7864
Epoch 3/10
375/375 [==============================] - ETA: 0s - loss: 0.5904 - accuracy: 0.8221
Epoch 3: val_accuracy improved from 0.78986 to 0.79907, saving model to smart_model_mobile.h5
375/375 [==============================] - 268s 715ms/step - loss: 0.5904 - accuracy: 0.8221 - val_loss: 0.7208 - val_accuracy: 0.7991
Epoch 4/10
375/375 [==============================] - ETA: 0s - loss: 0.5369 - accuracy: 0.8380
Epoch 4: val_accuracy improved from 0.79907 to 0.80128, saving model to smart_model_mobile.h5
375/375 [==============================] - 270s 720ms/step - loss: 0

## predict

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Rescale the image
    return img_array

# Example image path (update with the actual image path)
img_path = '/content/cropped/cropped/train/n02085620-Chihuahua/n02085620_10621.jpg'  # Update with the path to the image you want to predict

# Preprocess the image
img_array = preprocess_image(img_path)

# Make predictions using each trained model
for opt_name, model in trained_models.items():
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)

    class_labels = {v: k for k, v in train_data.class_indices.items()}  # Get class labels from the training data
    predicted_label = class_labels[predicted_class[0]]

    print(f'Predicted class with {opt_name} optimizer: {predicted_label}')


FileNotFoundError: [Errno 2] No such file or directory: '/content/cropped/cropped/train/n02085620-Chihuahua/n02085620_10621.jpg'

## Pratices example

In [ ]:
from keras.layers import Conv2D, BatchNormalization, ReLU, MaxPooling2D, Flatten, Dense
from keras.models import Sequential

def create_model():
  # Initialize the Sequential model
  model = Sequential()

  # First Convolutional Layer
  model.add(Conv2D(12, (3, 3), padding='same', input_shape=(224, 224, 3)))
  model.add(BatchNormalization())
  model.add(ReLU())
  model.add(MaxPooling2D(pool_size=(2, 2)))  # Reduces size to 12 × 75 × 75

  # Second Convolutional Layer
  model.add(Conv2D(20, (3, 3), padding='same'))
  model.add(BatchNormalization())
  model.add(ReLU())
  model.add(MaxPooling2D(pool_size=(2, 2)))  # Assuming no pooling for the second layer, size remains 20 × 75 × 75

  # Third Convolutional Layer
  model.add(Conv2D(32, (3, 3), padding='same'))
  model.add(BatchNormalization())
  model.add(ReLU())

  # Flatten the feature maps for the Fully Connected Layers
  model.add(Flatten())  # Converts 32 × 75 × 75 into a single 1D vector

  # Fully Connected Layers
  model.add(Dense(256, activation='relu'))  # Example dense layer for intermediate processing
  model.add(Dense(128, activation='relu'))  # Example dense layer for intermediate processing

  # Output Layer
  model.add(Dense(120, activation='softmax'))  # Output layer with 2 classes (healthy and unhealthy)

  return model


In [ ]:
def train_model(optimizer, optimizer_name):
    model = create_model()
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    history = model.fit(
        train_data,
        epochs=25,
        validation_data=test_data,
        class_weight=class_weights
    )

    # Evaluate the model
    loss, accuracy = model.evaluate(test_data)
    print(f'Test loss with {optimizer_name}: {loss*100:.4f}')
    print(f'Test accuracy with {optimizer_name}: {accuracy*100:.4f}')


    highest_val_accuracy = max(history.history['val_accuracy'])
    print(f'Highest Validation Accuracy with {optimizer_name}: {highest_val_accuracy * 100:.2f}%')

    return model, history

# List of optimizers
optimizers = {
    #'SGD': SGD(learning_rate=0.001),
    #'Adagrad': Adagrad(learning_rate=0.01),
    #'Adadelta': Adadelta(learning_rate=0.01),
    #'RMSProp': RMSprop(learning_rate=0.001),
    'Adam': Adam(learning_rate=0.001)
}

# Train and evaluate the model with different optimizers
# histories = {}
# for opt_name, opt in optimizers.items():
#     print(f"Training with {opt_name} optimizer")
#     histories[opt_name] = train_model(opt, opt_name)

trained_models = {}
histories = {}
for opt_name, opt in optimizers.items():
    print(f"Training with {opt_name} optimizer")
    model, history = train_model(opt, opt_name)
    trained_models[opt_name] = model
    histories[opt_name] = history


In [ ]:
def train_model(optimizer, optimizer_name):
    model = create_model()
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    history = model.fit(
        train_data,
        epochs=25,
        validation_data=test_data,
        class_weight=class_weights
    )

    # Evaluate the model
    loss, accuracy = model.evaluate(test_data)
    print(f'Test loss with {optimizer_name}: {loss*100:.4f}')
    print(f'Test accuracy with {optimizer_name}: {accuracy*100:.4f}')


    highest_val_accuracy = max(history.history['val_accuracy'])
    print(f'Highest Validation Accuracy with {optimizer_name}: {highest_val_accuracy * 100:.2f}%')

    return model, history

# List of optimizers
optimizers = {
    # 'SGD': SGD(learning_rate=0.001),
    # 'Adagrad': Adagrad(learning_rate=0.01),
    # 'Adadelta': Adadelta(learning_rate=0.01),
    # 'RMSProp': RMSprop(learning_rate=0.001),
    'Adam': Adam(learning_rate=0.0001)
}

# Train and evaluate the model with different optimizers
trained_models = {}
histories = {}
for opt_name, opt in optimizers.items():
    print(f"Training with {opt_name} optimizer")
    model, history = train_model(opt, opt_name)
    trained_models[opt_name] = model
    histories[opt_name] = history


Training with Adam optimizer
Epoch 1/25
102/102 [==============================] - 114s 1s/step - loss: 0.5127 - accuracy: 0.7691 - val_loss: 0.8734 - val_accuracy: 0.4983
Epoch 2/25
102/102 [==============================] - 122s 1s/step - loss: 0.3598 - accuracy: 0.8287 - val_loss: 0.7036 - val_accuracy: 0.6312
Epoch 3/25
102/102 [==============================] - 114s 1s/step - loss: 0.3007 - accuracy: 0.8648 - val_loss: 0.5378 - val_accuracy: 0.7243
Epoch 4/25
102/102 [==============================] - 111s 1s/step - loss: 0.2785 - accuracy: 0.8694 - val_loss: 0.4254 - val_accuracy: 0.7774
Epoch 5/25
102/102 [==============================] - 115s 1s/step - loss: 0.2322 - accuracy: 0.8910 - val_loss: 0.4150 - val_accuracy: 0.7807
Epoch 6/25
102/102 [==============================] - 119s 1s/step - loss: 0.2243 - accuracy: 0.8880 - val_loss: 0.3843 - val_accuracy: 0.8073
Epoch 7/25
102/102 [==============================] - 112s 1s/step - loss: 0.2051 - accuracy: 0.9040 - val_loss: 

In [ ]:
import pandas as pd
import numpy as np

optimizers=np.array(['SGD Optimizer','Adagrad Optimizer','Adadelta Optimizer','RMSProp Optimizer','Adam Optimizer'])
test1= np.array(['89.36%','93.68%','87.37%','80.73%','100%'])
testt2=np.array(['89.70%','96.01%','96.68%','82.39%','100%'])

df=pd.DataFrame({'Optimizers':optimizers,'Base_Paper_Accuracy':test1,'Obtained_Accuracy':testt2})
df

,Optimizers,Base_Paper_Accuracy,Obtained_Accuracy
0,SGD Optimizer,89.36%,89.70%
1,Adagrad Optimizer,93.68%,96.01%
2,Adadelta Optimizer,87.37%,96.68%
3,RMSProp Optimizer,80.73%,82.39%
4,Adam Optimizer,100%,100%
